In [ ]:
import numpy as np
import pandas as pd
import networkx as nx

In [ ]:
# sl=eval(input('请输入边长：'))
# fp=(eval(input('请输入旗点纵坐标（从0开始）：')), eval(input('请输入旗点横坐标（从0开始）：')))

# mat=[]
# for i in range(sl):
#     mat.append(input('请输入第{}行（空格分隔）：'.format(i+1)).split())

# mat=pd.DataFrame(mat)

# mat

In [ ]:
fp=(eval(input('请输入旗点纵坐标（从0开始）：')), eval(input('请输入旗点横坐标（从0开始）：')))

mat=pd.read_excel(r'目录/地图.xlsx', header=None)
sl=np.shape(mat)[0]

mat

In [ ]:
G=nx.Graph()

# 添加节点，为其设置颜色属性
for i in range(sl):
    for j in range(sl):
        G.add_node((i,j), color=mat.iloc[i,j])

# 添加边，设置边权
for i in range(sl):
    for j in range(sl):
        if j+1<=sl-1:
            w=0 if (G._node[(i,j)])['color']==(G._node[(i,j+1)])['color'] else 1
            G.add_edge((i,j), (i,j+1), weight=w)
        if i+1<=sl-1:
            w=0 if (G._node[(i,j)])['color']==(G._node[(i+1,j)])['color'] else 1
            G.add_edge((i,j), (i+1,j), weight=w)

# print(G._node)
# print(pd.DataFrame(G.edges(data=True)))

In [ ]:
def func(G, fp, seq):
    G1=G.copy()
    steps=np.inf
    counter=0
    
    # 依次执行操作序列中的操作
    for operation in seq:
        # 遍历所有节点
        for i in range(sl):
            for j in range(sl):
                # 若其对应的格子在旗点所在色块上，则将其变色
                if nx.dijkstra_path_length(G1, source=fp, target=(i,j))==0:
                    (G1._node[(i,j)])['color']=operation
                    
        # 遍历所有边
        for edge in list(G1.edges):
            # 将同色节点间的边权重设为0
            if (G1._node[edge[0]])['color']==(G1._node[edge[1]])['color']:
                (G1[edge[0]][edge[1]])['weight']=0
                
        counter+=1
        
        if len(set([(G1._node[p])['color'] for p in G1._node]))==1:
            steps=counter
            break
            
    return steps

In [ ]:
def seq_generator(G, fp, n, l):
    # 获取旗点颜色
    pre_color=(G._node[fp])['color']
    seq_mat=[]
    for i in range(n):
        seq_ls=[]
        for j in range(l):
            # 不能变成当前颜色
            colors={'r','o','y','g','b','p'}-set(pre_color)
            curr_color=np.random.choice(list(colors))
            seq_ls.append(curr_color)
            # 更新前一步变成的颜色
            pre_color=curr_color
        
        seq_mat.append(seq_ls)
        
    return pd.DataFrame(seq_mat)

In [ ]:
# 关卡图，旗点坐标，种群规模，操作序列长度，每轮死亡数，变异率，目标步数，最大迭代次数，是否返回日志
def GA(G, fp, seq_num, seq_len, death_num, prob_mut, target_steps, max_iter, return_log=False):
    seq_opt=[]
    min_steps=np.inf
    seq_df=seq_generator(G, fp, seq_num, seq_len)
    steps_of_curr_seqs=pd.DataFrame(np.empty((seq_num, 1), dtype=int), columns=['steps'])
    log=[]
    
    for i in range(max_iter+1):
        # 计算每个个体的适应度（通关操作步数）
        for j in range(seq_num):
            steps_of_curr_seqs.iloc[j,0]=func(G, fp, seq_df.iloc[j,:])
        
        # 将个体按通关操作步数从低到高排序
        df_sorted=pd.concat([seq_df, steps_of_curr_seqs], axis=1).sort_values(by='steps')
        # 将当前迭代次数与当前最小通关操作步数写入日志
        log.append([i, df_sorted['steps'].iloc[0]])
        
        # 判断最小通关操作步数是否有所下降
        if df_sorted['steps'].iloc[0]<min_steps:
            min_steps=df_sorted['steps'].iloc[0]
            seq_opt=list(df_sorted.iloc[0,:-1])
        
        # 判断是否达到目标操作步数
        if min_steps<=target_steps:
            break
        
        # 选出存活的个体
        seq_alive_df=df_sorted.iloc[0:seq_num-death_num, :-1]
        
        # 繁殖
        seq_new_ls=[]
        for j in range(death_num):
            # 选出新个体的亲代
            parents_index=np.random.choice(range(seq_num-death_num), 2)
            
            # 找出可选切割点
            cut_positions=[]
            for k in range(seq_len-1):
                if seq_alive_df.iloc[parents_index[0], k]!=seq_alive_df.iloc[parents_index[1], k+1]:
                    cut_positions.append(k)
            
            # 选出切割位置
            cut_position=np.random.choice(cut_positions) if cut_positions!=[] else np.random.choice(range(seq_len-1))
            
            # 生成新个体
            newseq_head=list(seq_alive_df.iloc[parents_index[0], :cut_position+1])
            newseq_tail=list(seq_alive_df.iloc[parents_index[1], cut_position+1:])
            seq_new_ls.append(newseq_head+newseq_tail)
        
        seq_new_df=pd.DataFrame(seq_new_ls)
        seq_df_new=pd.concat([seq_alive_df, seq_new_df], axis=0)
        
        # 选出突变个体
        mut_num=round(seq_num*prob_mut)
        mut_seq_index=np.random.choice(range(seq_num), mut_num)
        
        # 对突变个体执行突变
        for j in range(mut_num):
            mut_position=np.random.randint(0, seq_len)
            seq_df_new.iloc[mut_seq_index[j], mut_position]=np.random.choice(['r','o','y','g','b','p'])
        
        # 更新种群
        seq_df=seq_df_new
        # 重置操作序列DataFrame索引，防止其无法与通关步数DataFrame拼合
        seq_df.index=range(seq_num)
        
    if return_log==True:
        return min_steps, seq_opt, pd.DataFrame(log, columns=['iterations', 'curr_min_steps'])
    else:
        return min_steps, seq_opt

In [ ]:
# G, fp, seq_num, seq_len, death_num, prob_mut, target_steps, max_iter, return_log
min_steps, seq_opt, log=GA(G, fp, 200, 15, 50, 0.01, 0, 10, True)

print(min_steps)
print(seq_opt)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

plt.plot(log['iterations'], log['curr_min_steps'])
plt.xlabel('iterations')
plt.ylabel('curr_min_steps')